# 単眼深度推定モデルを学習する

# ハイパーパラメータ

In [1]:
frame_idx = [0, -1] # 隣接フレームの番号
epochs = 100
lr = 0.0004
batch_size = 32

# ネットワーク

学習するネットワークは現在のフレームの深度マップを推定するものと、フレーム間の姿勢の変化を推定するものの２つである。  
ここではそれぞれdepth netとpose netと呼ぶ。
- depth netは深度マップを推定するネットワーク
- pose netは回転（X軸、Y軸、Z軸）と並進（X, Y, Z）の合計6個の数値を推定するネットワーク

## depth net
ここではネットワークの詳細に関心はないたえ、`segmentation_models_pytorch`を使いU-Netを定義する。  
最新の深度推定モデルは複数の解像度の深度マップを推定するのが一般的だが、ここではGPUのメモリを節約するために、１枚のみとする

In [2]:
import segmentation_models_pytorch as smp
import torch

depth_net = smp.Unet("efficientnet-b0", in_channels=3, classes=1, activation=None)

# 入出力確認
depth_net(torch.zeros(1, 3, 224, 224))

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth


  0%|          | 0.00/20.4M [00:00<?, ?B/s]

tensor([[[[ -1.5092,  -3.5802,  -3.9052,  ...,  -5.9807,  -2.9136,  -0.5965],
          [ -2.2135,  -1.7661,  -1.1237,  ..., -13.6258,  -5.4054,  -0.3367],
          [ -2.0020,  -3.0601,  -2.6086,  ...,  -9.8803, -11.2230,  -5.5673],
          ...,
          [ -4.6031,  -4.3523,  -0.6034,  ...,   1.9895,   0.9651,  -1.8151],
          [ -4.6992,  -3.0708,   0.4737,  ...,   2.7389,  -2.0580,  -0.3783],
          [ -0.7423,  -1.1441,   2.3566,  ...,  -2.5374,  -1.5815,  -0.4001]]]],
       grad_fn=<ConvolutionBackward0>)

## pose net
pose netに関してもネットワークの詳細に関心がないため、`torchvision`の学習済みモデルを持ってきて、部分的にレイヤーを差し替える。
現在フレームと隣接フレームの２枚を入力し、その２枚の間で発生した姿勢の変化量を推定する。  
- 入力: 2枚の画像はチャンネルの次元で結合して合計で6チャンネルのテンソルとして入力を行う。
- 出力： 出力は並進(X軸、Y軸、Z軸)と回転(X軸、Y軸、Z軸)の6DoFである。

In [3]:
from torchvision.models import resnet18
from torch.nn import Linear, Conv2d

pose_net = resnet18(pretrained=True)
pose_net.conv1 = Conv2d(in_channels=3*2, out_channels=64, kernel_size=7, stride=2, padding=3, bias=False)
pose_net.fc = Linear(in_features=512, out_features=6)

# 初期はすべて0を出力したほうが都合がよいので、weightとbiasを0クリアしておく
# reproj_loss_ += torch.randn(reproj_loss_.shape).to(device=reproj_loss_.device) * 1e-3
pose_net.fc.weight.data = torch.randn(pose_net.fc.weight.shape) * 1e-5
pose_net.fc.bias.data = torch.randn(pose_net.fc.bias.shape) * 1e-5

# 入出力確認
pose_net(torch.zeros(1, 3*2, 224, 224))

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

tensor([[ 3.0028e-04, -3.8320e-04, -4.6174e-05, -9.7556e-07, -1.4228e-04,
         -4.3509e-04]], grad_fn=<AddmmBackward0>)

# Loss関数
ここでは、Phtometric lossとSmoothness lossを定義する。  
Photometric lossは推定した深度と姿勢変化からSourceの画像をTargetの画像に合わせて一致しているかどうかをl1とSSIMで測る。  
また、[Monodepth2](https://arxiv.org/abs/1806.01260)で提案されたOcculusionや動体から生じる原理的に復元不可能な画素に対するLossの計算を回避するauto-maskingを導入した。  
Smoothness lossはPhotometric lossの復元誤差が濃淡が平滑な領域で勾配を得にくいという問題を解決するために導入した。  
このlossは近接ピクセルがおおよそ同じような深度を持っている（物体の境界以外は）という仮定のもと、それをLossとして与えるものである。  
実装は様々あるが、ここではX,Y方向の差分のみから計算する比較的にシンプルなものを用いる。  

In [4]:
import sys
sys.path.append("../src")

In [6]:
import torch.nn.functional as F
# from camera import PinholeCamera
from functools import lru_cache
import torchgeometry as tgm


class PhotometricLoss(torch.nn.Module):
    
    def __init__(self, frame_inds, weights=[0.6, 0.4], automasking=False):
        super().__init__()
        self.frame_inds = frame_inds
        self.l1_loss = torch.nn.L1loss(reduction="none")
        self.ssim_loss = tgm.losses.SSIM(reduction="none", window_size=3)
        self.weights = weights
        self.automasking = automasking
        
    def forward(self, y, y_pred):
        image_target = y["rgb_0"]
        depth = y_pred["depth_0"]
        intrinsic = y["intrinsic_0"]
        
        reproj_loss = []
        image_warped = {}
        for idx in self.frame_inds:
            if idx == 0: # ターゲット同士は比較しない
                continue
            image_source = y["rgb_{idx}"]
            extrinsic_src2tgt = y_pred["extrinsic_{idx}"]
            image_warped_ = self.warp(image_source, depth, intrinsic, extrinsic_src2tgt)
            l1_loss = self.l1_loss(image_warped_, image_target)
            ssim_loss = self.ssim_loss(image_warped_, image_target)
            reproj_loss = l1_loss * self.weights[0] + ssim_loss * self.weights[1]
            reproj_loss = torch.mean(reproj_loss_, dim=1) # auto-maskingで扱いやすいようにチャンネルの次元を潰しておく
            reproj_loss.append(reproj_loss_)
            image_warped[idx] = image_warped_
            
        if self.automasking:
            # auto-masking (https://arxiv.org/pdf/1806.01260.pdf) 何も変更を加えないSource画像を利用する。
            for idx in self.frame_inds:
                if idx == 0: # ターゲット同士は比較しない
                    continue
                image_source = y["rgb_{idx}"]
                l1_loss = self.l1_loss(image_source, image_target)
                ssim_loss = self.ssim_loss(image_source, image_target)
                reproj_loss_ = l1_loss * self.weights[0] + ssim_loss * self.weights[1]
                # 平坦な領域ではWarpされたものと何も変更を加えないものでLossが全くおなじになってしまう画素が生じる可能性があるので微小な乱数を加える
                reproj_loss_ += torch.randn(reproj_loss_.shape).to(device=reproj_loss_.device) * 1e-3
                reproj_loss_ = torch.mean(reproj_loss_, dim=1) # auto-maskingで扱いやすいようにチャンネルの次元を潰しておく
                reproj_loss.append(reproj_loss_)
                
            reproj_loss = torch.stack(reproj_loss, dim=1)
            loss, min_inds = torch.min(reproj_loss, dim=1)
            automask = (min_inds >= (reproj_loss.shape[1] // 2)).float()
            loss = reproj_loss.mean()
        else:
            loss = torch.stack(reproj_loss, dim=1)
            automask = torch.zeros_like(loss).squeeze(1) # dummy
            loss = loss.mean()
            
        return loss, image_warped, automask
        